# Descubra quem fez o ENEM apenas para treino

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
#import scikitplot as skplt

%matplotlib inline

In [2]:
train = pd.read_csv('train.csv', index_col=0)
train.shape

(13730, 166)

In [3]:
test  = pd.read_csv('test.csv')
test.shape

(4570, 43)

O dataset de teste possui menos colunas que o de treino. Desse modo, as colunas a mais serão descartadas e o target ('IN_TREINEIRO') mantido.

In [4]:
col = list(test.columns)
col.append('IN_TREINEIRO')

train = train[col]
train.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,IN_TREINEIRO
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,43,RS,24,M,1,1,1,4,1,...,520.0,D,D,C,A,A,C,H,A,0
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,23,CE,17,F,3,1,2,0,2,...,580.0,A,A,B,A,A,A,NaN,A,0
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,23,CE,21,F,3,1,3,0,1,...,NaN,D,D,C,A,A,A,NaN,A,0
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,33,RJ,25,F,0,1,1,9,1,...,NaN,H,E,E,C,B,C,F,D,0
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,13,AM,28,M,2,1,1,4,1,...,NaN,E,D,C,A,A,B,F,A,0


## Análise do dataset e seleção de parâmetros

In [5]:
train.isna().sum()/len(train)

NU_INSCRICAO              0.000000
CO_UF_RESIDENCIA          0.000000
SG_UF_RESIDENCIA          0.000000
NU_IDADE                  0.000000
TP_SEXO                   0.000000
TP_COR_RACA               0.000000
TP_NACIONALIDADE          0.000000
TP_ST_CONCLUSAO           0.000000
TP_ANO_CONCLUIU           0.000000
TP_ESCOLA                 0.000000
TP_ENSINO                 0.688128
TP_DEPENDENCIA_ADM_ESC    0.688128
IN_BAIXA_VISAO            0.000000
IN_CEGUEIRA               0.000000
IN_SURDEZ                 0.000000
IN_DISLEXIA               0.000000
IN_DISCALCULIA            0.000000
IN_SABATISTA              0.000000
IN_GESTANTE               0.000000
IN_IDOSO                  0.000000
TP_PRESENCA_CN            0.000000
TP_PRESENCA_CH            0.000000
TP_PRESENCA_LC            0.000000
TP_PRESENCA_MT            0.000000
NU_NOTA_CN                0.246832
NU_NOTA_CH                0.246832
NU_NOTA_LC                0.261981
TP_LINGUA                 0.000000
TP_STATUS_REDACAO   

Nesta primeira análise, todas as colunas que possuem algum dado faltante serão descartadas.

In [6]:
train.drop(train.columns[train.isna().sum()/len(train) > 0], axis=1, inplace=True)

### Matriz de correlação

In [7]:
train.corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'),axis=1)

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,TP_LINGUA,IN_TREINEIRO
CO_UF_RESIDENCIA,1.0,-0.002,-0.23,0.0094,0.026,-0.033,0.041,0.0014,nan,-0.011,0.0037,0.0011,-0.03,-0.0095,-0.014,-0.035,-0.035,-0.034,-0.034,-0.096,-0.035
NU_IDADE,-0.002,1.0,0.0097,-0.018,-0.25,0.68,-0.3,0.028,nan,0.0065,0.00049,-0.0044,0.005,0.0065,0.074,-0.25,-0.25,-0.24,-0.24,0.082,-0.3
TP_COR_RACA,-0.23,0.0097,1.0,0.016,0.00074,0.0039,-0.042,-0.015,nan,0.0079,-0.01,0.0069,0.017,-0.01,0.0098,-0.024,-0.024,-0.022,-0.022,0.12,0.0097
TP_NACIONALIDADE,0.0094,-0.018,0.016,1.0,-5.4e-05,-0.026,0.024,0.0094,nan,0.011,-0.0016,-0.0016,0.013,-0.0058,-0.0023,-0.02,-0.02,-0.024,-0.024,0.015,-0.015
TP_ST_CONCLUSAO,0.026,-0.25,0.00074,-5.4e-05,1.0,-0.59,0.16,0.018,nan,0.013,-0.0076,0.0021,-0.023,-0.0085,-0.011,0.055,0.055,0.053,0.053,-0.025,0.53
TP_ANO_CONCLUIU,-0.033,0.68,0.0039,-0.026,-0.59,1.0,-0.42,-0.00096,nan,-0.012,0.0049,-0.0057,0.012,0.0089,0.024,-0.22,-0.22,-0.22,-0.22,0.062,-0.26
TP_ESCOLA,0.041,-0.3,-0.042,0.024,0.16,-0.42,1.0,-0.0075,nan,0.0016,-0.0054,0.025,-0.008,-0.011,-0.0076,0.17,0.17,0.18,0.18,-0.07,-0.24
IN_BAIXA_VISAO,0.0014,0.028,-0.015,0.0094,0.018,-0.00096,-0.0075,1.0,nan,-0.00067,-0.0003,-0.0003,-0.0035,-0.0011,-0.00042,-0.0088,-0.0088,-0.0073,-0.0073,0.0038,-0.014
IN_CEGUEIRA,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
IN_SURDEZ,-0.011,0.0065,0.0079,0.011,0.013,-0.012,0.0016,-0.00067,nan,1.0,-0.00016,-0.00016,-0.0019,-0.00059,-0.00023,0.002,0.002,0.0027,0.0027,0.00025,-0.0074


Apenas as colunas que possuem correlação maior que 20% (positiva ou negativa) com o target serão selecionadas para treinamento do modelo.

In [8]:
pearsoncorr = train.corr(method='pearson')
var = pearsoncorr.columns[(pearsoncorr.IN_TREINEIRO>0.20) | (pearsoncorr.IN_TREINEIRO<-0.20)] 
var = var.tolist()
var

['NU_IDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO']

In [9]:
train = train[var]
train.head()

,NU_IDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO
1,24,1,4,1,0
2,17,2,0,2,0
3,21,3,0,1,0
4,25,1,9,1,0
5,28,1,4,1,0


Criação do dataframe de resposta e aplicando as modificações nos dados de teste:

In [10]:
answer = pd.DataFrame()
answer['NU_INSCRICAO'] = test['NU_INSCRICAO']

test = test[train.columns[:-1]]
test.head()

,NU_IDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA
0,19,1,3,1
1,24,1,4,1
2,16,3,0,1
3,17,2,0,2
4,19,1,1,1


## Treinamento do classificador 

In [11]:
y = train['IN_TREINEIRO']
X = train.drop(['IN_TREINEIRO'], axis=1)

clf = RandomForestClassifier(random_state=0).fit(X, y)

### Resposta final: 99.74%

In [12]:
pred = clf.predict(test)

answer['IN_TREINEIRO'] = pred
answer.head()

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0


In [13]:
answer.to_csv('answer.csv',index=False)